In [ ]:
from pyincore import IncoreClient, RepairService, MappingSet, FragilityService
from pyincore.analyses.commercialbuildingrecovery.commercialbuildingrecovery import CommercialBuildingRecovery
from pyincore.analyses.buildingdamage import BuildingDamage

import pyincore.globals as pyglobals

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

### Joplin Commercial Building Recovery.

In [ ]:
# Joplin
buildings = "5dbc8478b9219c06dd242c0d"  # ergo:buildingInventoryVer6 5dbc8478b9219c06dd242c0d

# Create commercial recovery instance
com_recovery = CommercialBuildingRecovery(client)
com_recovery.load_remote_input_dataset("buildings", buildings)


In [ ]:
fragility_service = FragilityService(client)
mapping_id = "5d8942dfb9219c068ea795ea"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [ ]:
# Perform building damage analysis
bldg_dmg = BuildingDamage(client)
tornado_id = "5d07cbeab9219c065b080930"

# setting input data and parameters
bldg_dmg.load_remote_input_dataset("buildings", buildings ) 
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set )
bldg_dmg.set_parameter("hazard_type", "tornado")
bldg_dmg.set_parameter("hazard_id",tornado_id )
bldg_dmg.set_parameter("num_cpu", 4)
bldg_dmg.set_parameter("result_name", "joplin_tornado_commerical_bldg_dmg")
bldg_dmg.run_analysis()
building_dmg_result = bldg_dmg.get_output_dataset("ds_result")

In [ ]:
# Recovery mapping
mapping_id = "60edfa3efc0f3a7af53a21b5"
# Create repair service
repair_service = RepairService(client)
mapping_set = MappingSet(repair_service.get_mapping(mapping_id))
com_recovery.set_input_dataset('dfr3_mapping_set', mapping_set)

In [ ]:
# input datsets ids
sample_damage_states = "651dcf587faf5a0eae9e3443"  # 10 samples 28k buildings - MCS output format
mcs_failure = '651dced2adce7a5cb977f06e'
delay_factors = "64ee0bcd553ecf0768e21e55"

In [ ]:
# Load input datasets
com_recovery.load_remote_input_dataset("sample_damage_states", sample_damage_states)
com_recovery.load_remote_input_dataset("mcs_failure", mcs_failure)
com_recovery.load_remote_input_dataset("delay_factors", delay_factors)
com_recovery.set_input_dataset("building_dmg", building_dmg_result)

In [ ]:
# Input parameters
num_samples = 10

In [ ]:
# Specify the result name
result_name = "joplin_commercial_test"

# Set analysis parameters
com_recovery.set_parameter("result_name", result_name)
com_recovery.set_parameter("num_samples", num_samples)

In [ ]:
# Run the analysis (NOTE: with SettingWithCopyWarning)
com_recovery.run_analysis()

In [ ]:
# Retrieve result dataset
result = com_recovery.get_output_dataset("time_stepping_recovery")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()